# LLMs

Here we'll be interacting with a server that's exposing 2 LLMs via the runnable interface.

In [1]:
from langchain.prompts.chat import ChatPromptTemplate

In [2]:
from langserve import RemoteRunnable

openai_llm = RemoteRunnable("http://localhost:8000/openai/")


In [3]:
prompt = "Tell me a joke about AI"
response = openai_llm.invoke(prompt)
print(response)


content='Why did the AI break up with its robot girlfriend?\n\nBecause she had too many bugs in her code!' response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4469bcbf-cd4d-4bb9-8aad-5ea041aa6a3d-0' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34}


Let's create a prompt composed of a system message and a human message.

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences.",
        ),
        ("human", "Tell me about your favorite novel"),
    ]
).format_messages()

We can use either LLM

In [ ]:
openai_llm.invoke(prompt)

As with regular runnables, async invoke, batch and async batch variants are available by default

In [6]:
await openai_llm.ainvoke(prompt)

AIMessage(content='My favorite novel is "Ulysses" by James Joyce. It\'s a complex and innovative work that explores the intricacies of human consciousness and the challenges of modernity in a highly poetic and experimental manner. The prose is richly layered and rewards careful reading.', additional_kwargs={}, example=False)

Streaming is available by default

In [5]:
async for chunk in anthropic.astream(prompt):
    print(chunk.content, end="", flush=True)

 My favorite novel is The Art of Language by Maximo Quilana. It is a philosophical treatise on the beauty and complexity of human speech. The prose is elegant yet precise.

In [13]:
from langchain.schema.runnable import RunnablePassthrough

In [14]:
comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | openai_llm
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | anthropic
)


chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [15]:
chain.invoke({})

{'joke': AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, example=False),
 'classification': AIMessage(content=" not funny\nWhy don't scientists trust atoms?", additional_kwargs={}, example=False)}